In [23]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly

In [68]:
# Importação dos daddos
df_selic_serie_historica=pd.read_csv('https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=csv', sep=';')
df_selic_serie_historica.to_csv('./output/selic_serie_historica_df.csv')


In [24]:
# Parse de tratamento dos dados
df_selic=pd.read_csv('./output/selic_serie_historica_df.csv')
df_selic=df_selic.drop(columns=['Unnamed: 0'])
df_selic['data']=df_selic.apply(lambda x: datetime.strptime(x.data,'%d/%m/%Y').date(),axis=1)
df_selic=df_selic.set_index('data')
df_selic['valor']=df_selic.apply(lambda x: float(x.valor.replace(',','.')),axis=1)
df_selic=df_selic.sort_index(ascending=False)

In [25]:
1.1365**(1/252)-1

0.000507880373261882

In [40]:
def converte_ad_aa(taxa_ad):
    # recebi taxa ao dia em porcentagem
    taxa_ad=taxa_ad/100                 #tira porcentagem e coloca em fração
    #juros sobre juros e considera 252 dias
    return round(((1+taxa_ad)**(252)-1)*100,2)

In [41]:
df_selic['selic_aa']=df_selic.apply(lambda x: converte_ad_aa(x.valor),axis=1)
df_selic.head(36)

,valor,selic_aa
data,,
2022-08-29,0.050788,13.65
2022-08-26,0.050788,13.65
2022-08-25,0.050788,13.65
2022-08-24,0.050788,13.65
2022-08-23,0.050788,13.65
2022-08-22,0.050788,13.65
2022-08-19,0.050788,13.65
2022-08-18,0.050788,13.65
2022-08-17,0.050788,13.65


In [27]:
# Gráfico iterativo utilizando o plotly
fig=px.line(x=df_selic.index,y=df_selic['selic_aa'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/selic_serie_historica_grafico.html')
# fig.show()

'./output/selic_serie_historica_grafico.html'

In [29]:
valor_periodo=df_selic.loc[datetime.strptime('2022-08-03','%Y-%m-%d').date():datetime.strptime('2022-06-17','%Y-%m-%d').date(),'valor'].sum()
valor_periodo
# 17/06/2022 - 03/08/2022

1.667258

In [31]:

dias=len(df_selic.loc[datetime.strptime('2022-08-03','%Y-%m-%d').date():datetime.strptime('2022-06-17','%Y-%m-%d').date(),'valor'])